In [ ]:
import h5py
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
with h5py.File('train.h5', mode='r') as f:
    print(f['matrix'].shape)
    data_len = f['matrix'].shape[0]

with h5py.File('valid.h5', mode='r') as f:
    valid_data_len = f['matrix'].shape[0]

In [ ]:
epochs = 500
load_batch = 1024
batch = 4
iter_train = data_len // batch
iter_valid = valid_data_len // batch

In [ ]:
def generator(file):
    with h5py.File(file, mode='r') as src:
        data_len = src['matrix'].shape[0]
        batch_indexes = np.array(list(range(int(np.ceil(data_len / load_batch)))))
        for e in range(epochs):
            np.random.shuffle(batch_indexes)
            for i in batch_indexes:
                slc = slice(i, i + load_batch if i + load_batch < data_len else data_len)
                with tf.device('/GPU:0'):
                    # seleciona somente canais de infravermelho e microondas
                    images = src['matrix'][slc, :, :, [0, 2]]
                    # images = src['matrix'][slc]
                    images = np.nan_to_num(images, copy=False)
                    images[images > 1000] = 0.0
                    avgs = tf.math.reduce_mean(images, axis=[0, 1, 2])
                    stds = tf.math.reduce_std(images, axis=[0, 1, 2])
                    norm = (images - avgs) / stds
                    img_height = img_width = 201
                    img_crop_w = 64
                    st1 = (img_height // 2 - img_crop_w // 2)
                    height_crop = slice(st1, st1 + img_crop_w)
                    st2 = (img_width // 2 - img_crop_w // 2)
                    width_crop = slice(st2, st2 + img_crop_w)
                    norm = norm[:, height_crop, width_crop, :]
                info = src['info'][slc]
                info = tf.convert_to_tensor(info)
                for j in range(norm.shape[0]):
                    yield norm[j], tf.expand_dims(info[j], axis=0)

In [ ]:
train_generator = generator('train.h5')
valid_generator = generator('valid.h5')
test_generator = generator('test.h5')

img_w = 64
output_signature = (
    tf.TensorSpec(shape=(img_w, img_w, 3), dtype=tf.float32),
    tf.TensorSpec(shape=(1,), dtype=tf.float32)
)

train_ds = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=output_signature
).batch(1)

valid_ds = tf.data.Dataset.from_generator(
    lambda: valid_generator,
    output_signature=output_signature
).batch(1)

test_ds = tf.data.Dataset.from_generator(
    lambda: test_generator,
    output_signature=output_signature
).batch(1)

In [ ]:

from tensorflow import keras

def build_model(input_shape, strides=(2, 2)):
    initializer = keras.initializers.RandomNormal(mean=0.0, stddev=0.01)
    reg = keras.regularizers.L2(1e-5)
    model = keras.models.Sequential()
    model.add(keras.layers.Input(input_shape))
    model.add(keras.layers.Conv2D(16, (4, 4), strides=strides, activation='relu', kernel_initializer=initializer, bias_initializer=initializer, kernel_regularizer=reg, bias_regularizer=reg))
    model.add(keras.layers.Conv2D(32, (3, 3), strides=strides, activation='relu', kernel_initializer=initializer, bias_initializer=initializer, kernel_regularizer=reg, bias_regularizer=reg))
    model.add(keras.layers.Conv2D(64, (3, 3), strides=strides, activation='relu', kernel_initializer=initializer, bias_initializer=initializer, kernel_regularizer=reg, bias_regularizer=reg))
    model.add(keras.layers.Conv2D(128, (3, 3), strides=strides, activation='relu', kernel_initializer=initializer, bias_initializer=initializer, kernel_regularizer=reg, bias_regularizer=reg))
    
    model.add(keras.layers.Flatten())
    
    model.add(keras.layers.Dense(256, activation='relu', kernel_initializer=initializer, kernel_regularizer=reg, bias_initializer=initializer, bias_regularizer=reg))
    model.add(keras.layers.Dense(64, activation='relu', kernel_initializer=initializer, kernel_regularizer=reg, bias_initializer=initializer, bias_regularizer=reg))
    model.add(keras.layers.Dense(1, activation='linear', kernel_initializer=initializer, kernel_regularizer=reg, bias_initializer=initializer, bias_regularizer=reg))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=.0005), loss='mse', metrics=['mse'])
    return model

In [ ]:
model = build_model((64, 64, 3),)
model.summary()

In [ ]:
with tf.device('/GPU:0'):
    model.fit(
        train_ds,
        validation_data=valid_ds,
        epochs=epochs,
        steps_per_epoch=iter_train,
        validation_steps=iter_valid
    )

model.save('result.h5')